In [26]:
import sys
import os
import glob
from dask.array.image import imread as daskread
from pathlib import Path
import pandas as pd
import panel as pn
import colorcet as cc
import hvplot.pandas 
import xarray as xr
import rasterio as rs
import holoviews as hv
import hvplot.xarray  # noqa


In [27]:
dashboard_title = pn.panel('ONEAT')

header = pn.Row(dashboard_title)
header

Row
    [0] Markdown(str)

In [29]:
from tifffile import imwrite, imread
filepath = '/Users/aimachine/LightSheetTraining/ValRaw/Astec-Pm2_fuse_t001.tif'

ds = xr.open_rasterio(filepath)

csvname = '/Users/aimachine/oneatgolddust/Test/Raw/Saved29s4resf32/MacroCheateLocationwt_gold_mov12_SW30.csv'
print('Timelapse image from rasterio',ds)

ds.sel(band=1).hvplot.image(cmap='Greys_r', rasterize=True, data_aspect=1)

/Users/aimachine/opt/anaconda3/envs/tensorflowGPU/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Timelapse image from rasterio <xarray.DataArray (band: 1, y: 482, x: 533)>
[256906 values with dtype=uint16]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 477.5 478.5 479.5 480.5 481.5
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 528.5 529.5 530.5 531.5 532.5
Attributes:
    transform:                 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0)
    res:                       (1.0, -1.0)
    is_tiled:                  0
    nodatavals:                (nan,)
    scales:                    (1.0,)
    offsets:                   (0.0,)
    TIFFTAG_IMAGEDESCRIPTION:  ImageJ=1.52p\nimages=670\nslices=670\nunit=pix...
    TIFFTAG_RESOLUTIONUNIT:    1 (unitless)
    TIFFTAG_XRESOLUTION:       3.3333333
    TIFFTAG_YRESOLUTION:       3.3333333


:DynamicMap   []
   :Image   [x,y]   (value)

In [ ]:
dataset   = pd.read_csv(csvname, delimiter = ',')
#Data is written as T, Y, X, Score, Size, Confidence
T = dataset[dataset.keys()[0]][1:].astype('int16')
Y = dataset[dataset.keys()[1]][1:]
X = dataset[dataset.keys()[2]][1:]
Score = dataset[dataset.keys()[3]][1:]
Size = dataset[dataset.keys()[4]][1:]
Confidence = dataset[dataset.keys()[5]][1:]
print((dataset.shape[0]))
i = 0
condition = dataset["T"] == i
limitT = T[condition]
limitX = X[condition]
limitY = Y[condition]
limitScore = Score[condition]
limitSize = Size[condition]
limitConfidence = Confidence[condition]
timed_dataset = pd.DataFrame(list(zip(limitT,limitY,limitX,limitScore,limitConfidence)), index = None, 
                                                                      columns =['T', 'Y', 'X', 'Score', 'Confidence'])
print(timed_dataset)
bands = list(range(0, timed_dataset.shape[0])) 
print(bands)
bands = [xr.open_rasterio(file_path%band).load()[0] for band in bands]


In [ ]:
print(dataset.shape)
dataset.head()

In [ ]:
print(timed_dataset.shape)
timed_dataset.head()

In [ ]:
sample_points = timed_dataset.hvplot.points(x='X', y='Y', c='Score', cmap=cc.CET_L4, responsive=True)
rasterized_pop = cleaned_ds.hvplot.image(responsive=True)
detection = rasterized_pop * sample_points
print(detection)

In [ ]:
mini_dashboard = pn.Column(header, detection)
mini_dashboard

In [ ]:
mini_dashboard.show()
